In [1]:
!pip install chromadb openai pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00:00


In [2]:
!pip install -q langgraph pydantic chromadb openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 14.8 MB/s eta 0:00:00


In [11]:
!pip install tavily-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00


In [4]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import pandas as pd
from google.colab import userdata  # ✅ Secure key access

# ✅ Load API Key securely from Colab
key = userdata.get('OPENAI_API_KEY')

# Load CSVs
sales_df = pd.read_csv("/content/sales.csv")
customers_df = pd.read_csv("/content/customers.csv")
campaign_df = pd.read_csv("/content/marketing_campaign.csv")

# Create new Chroma client using the new method (no Settings needed)
client = chromadb.PersistentClient(path="./chroma_db")

# Define embedding function with secure API key
embedding_function = OpenAIEmbeddingFunction(api_key=key)

# Create or get collection
collection = client.get_or_create_collection(
    name="campaign_data",
    embedding_function=embedding_function
)

# Chunking functions
def chunk_sales(row):
    return f"On {row['sale_date']}, a {row['fuel_variant']} {row['model']} ({row['cartype']} - {row['model_variant']} variant) was sold in {row['city']} ({row['region']} region) for ₹{row['sale_amount']}."

def chunk_customers(row):
    return f"Customer {row['customer_id']} is a {row['age']}-year-old {row['gender']} who prefers {row['preferred_fuel_variant']} {row['preferred_model']} ({row['preferred_cartype']} - {row['preferred_model_variant']} variant). Purchase type: {row['purchase_type']}."

def chunk_campaigns(row):
    return f"{row['marketing_channel']} campaign '{row['campaign_name']}' targeting {row['target_cartype']}s for {row['target_audience']} offered a {row['discount_percent']}% discount and had {row['actual_sales']} actual sales."

# Create text chunks
sales_chunks = [chunk_sales(row) for _, row in sales_df.iterrows()]
customer_chunks = [chunk_customers(row) for _, row in customers_df.iterrows()]
campaign_chunks = [chunk_campaigns(row) for _, row in campaign_df.iterrows()]

# Combine all chunks
all_chunks = sales_chunks + customer_chunks + campaign_chunks

# Add to ChromaDB collection
collection.add(
    documents=all_chunks,
    ids=[f"doc_{i}" for i in range(len(all_chunks))]
)

print("✅ Chunks embedded and saved to ChromaDB successfully.")


✅ Chunks embedded and saved to ChromaDB successfully.


In [6]:
import chromadb

# Connect to ChromaDB
client = chromadb.PersistentClient(path="./chroma_db")

# List available collection names (new method)
collections = client.list_collections()
print("Available collections:", collections)

# Connect to the existing collection
collection = client.get_collection(name="campaign_data")

# Fetch and print some stored documents
results = collection.get(
    include=["documents"],
    limit=2000
)

print("\nStored Chunks in 'campaign_data':\n")
for i, doc in enumerate(results["documents"]):
    print(f"{i+1}. {doc}")


Available collections: [Collection(name=campaign_data)]

Stored Chunks in 'campaign_data':

1. On 2023-11-27, a Petrol Tata Harrier (SUV - Base variant) was sold in Varanasi (North region) for ₹4600000.
2. On 2023-12-22, a Diesel Kia Seltos (SUV - Mid variant) was sold in Ahmedabad (West region) for ₹3900000.
3. On 2023-01-20, a Petrol Skoda Slavia (Sedan - Top variant) was sold in Darjeeling (East region) for ₹4000000.
4. On 2023-03-15, a Diesel Tata Harrier (SUV - Second-Top variant) was sold in Mysuru (South region) for ₹1800000.
5. On 2023-03-15, a Diesel Toyota Fortuner (SUV - Top variant) was sold in Mysuru (South region) for ₹1500000.
6. On 2023-03-17, a Petrol Tata Harrier (SUV - Base variant) was sold in Ahmedabad (West region) for ₹4200000.
7. On 2023-08-22, a Petrol Tata Harrier (SUV - Base variant) was sold in Kolkata (East region) for ₹2000000.
8. On 2023-01-23, a Petrol MG Hector (SUV - Top variant) was sold in Bangalore (South region) for ₹4400000.
9. On 2023-01-07, a Di

In [7]:


from pydantic import BaseModel
from typing import Optional, Any
import re
import chromadb
from langgraph.graph import StateGraph
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from google.colab import userdata
from openai import OpenAI
from datetime import datetime


key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=key)

# Shared State
class CampaignState(BaseModel):
    goal: str
    parsed_goal: Optional[dict] = None
    vector_db: Optional[Any] = None
    market_analysis: Optional[str] = None
    audience_segments: Optional[str] = None
    campaign_strategy: Optional[str] = None
    campaign_content: Optional[str] = None
    campaign_simulation: Optional[str] = None
    campaign_report: Optional[str] = None

# ✍️ Agent 1: Parse Goal
def parse_goal(state: CampaignState) -> CampaignState:
    text = state.goal
    pattern = re.compile(r"(?i)(boost|increase|grow|improve)\s+(\w+)\s+sales\s+in\s+(\w+)\s+for\s+the\s+(.+)", re.IGNORECASE)
    match = pattern.search(text)
    if match:
        state.parsed_goal = {
            "goal": "Boost sales",
            "cartype": match.group(2).upper(),
            "region": match.group(3).capitalize(),
            "campaign_name": match.group(4).strip().title()
        }
    else:
        state.parsed_goal = {"error": "Unable to parse goal."}
    return state

# 🔍 Agent 2: Research Data from ChromaDB
def research_data(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal
    vector_db = state.vector_db
    if not parsed or not vector_db or "cartype" not in parsed:
        state.market_analysis = "Missing or invalid parsed goal."
        return state
    query = f"{parsed['cartype']} sales and campaign trends in {parsed['region']} for {parsed['campaign_name']} campaign"
    results = vector_db.query(query_texts=[query], n_results=5)
    state.market_analysis = "\n\n".join(results["documents"][0])
    return state

# 👥 Agent 3: Segment Audience
def segment_audience(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal
    vector_db = state.vector_db
    if not parsed or not vector_db or "cartype" not in parsed:
        state.audience_segments = "Missing or invalid parsed goal."
        return state
    query = f"Customers in {parsed['region']} who prefer {parsed['cartype']} cars"
    results = vector_db.query(query_texts=[query], n_results=5)
    state.audience_segments = "\n\n".join(results["documents"][0])
    return state

# 📈 Agent 4: Plan Strategy
def plan_strategy(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal or {}
    research = state.market_analysis
    audience = state.audience_segments
    if not parsed or not research or not audience or "region" not in parsed or "cartype" not in parsed:
        state.campaign_strategy = "Missing inputs for planning strategy."
        return state
    campaign = parsed.get("campaign_name", "Unnamed Campaign")
    region = parsed.get("region", "Unknown")
    cartype = parsed.get("cartype", "Vehicle")
    plan = f"""
📋 Campaign Strategy for '{campaign}':

- 📍 Region: {region}
- 🚗 Cartype: {cartype}
- 📋 Target Audience: Based on segment data, target high-converting buyer groups.
- 📣 Channels: Based on research, use SMS + Email
- 💰 Budget Estimate: ₹10,00,000
- 🍿 Discount: Offer 15% off
- ⏳ Duration: 15 days
- 🎯 KPIs:
  - Sales Target: 150 units
  - Engagement Rate: 25%
    """
    state.campaign_strategy = plan.strip()
    return state

# ✨ Agent 5: Generate Campaign Content
def generate_content(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal
    strategy = state.campaign_strategy
    audience = state.audience_segments

    if not parsed or not strategy or not audience or "cartype" not in parsed:
        state.campaign_content = "Missing data for content generation."
        return state

    try:
        campaign = parsed.get("campaign_name", "Campaign")
        cartype = parsed.get("cartype", "Car")
        region = parsed.get("region", "Your Area")

        content = f"""
📢 Marketing Content for '{campaign}' Campaign:

📱 SMS (3 variants):
1. 🚗 Big Savings on {cartype}s! Get exclusive {campaign} deals in {region}. Limited time!
2. 🔥 {campaign} is live! Grab 15% off on your dream {cartype}. Hurry!
3. 🎉 Special Offer on {cartype}s in {region}! Celebrate with 15% OFF!

✉️ Email:
Subject: Don’t Miss Our {campaign} Deals on {cartype}s!
Body: Celebrate this season with amazing offers on {cartype}s in {region}. Limited-time 15% discount, tailored for you! Visit our showroom or book online now!

📣 Social Media (3 posts):
1. 🎉 #{campaign.replace(" ", "")} – Get 15% OFF on {cartype}s in {region}. Drive festive joy home! 🚗✨
2. 🔥 #{campaign.replace(" ", "")} is here! Unbeatable offers on {cartype}s! Book now.
3. 📢 Exclusive {cartype} discounts for {region} customers. Festival + Fuel = Fun!

🌐 Landing Page:
Headline: "Your {cartype}, Now 15% Off – {campaign} is Here!"
Bullet Points:
- Limited-time festive discount
- Available in {region}
- Top {cartype} brands included
        """
        state.campaign_content = content.strip()
    except Exception as e:
        state.campaign_content = f"Content generation failed: {str(e)}"

    return state

# 🔮 Agent 6: Simulate Campaign Performance
def simulate_campaign(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal
    strategy = state.campaign_strategy
    audience = state.audience_segments
    content = state.campaign_content

    if not parsed or not strategy or not audience or not content:
        state.campaign_simulation = "Simulation skipped: Missing required inputs."
        return state

    prompt = f"""
Based on the following campaign goal and plan, simulate expected marketing performance:

Goal: {parsed}

Strategy:
{strategy}

Audience Insights:
{audience}

Content:
{content}

Give realistic estimates for:
- Estimated Reach
- Engagement Rate
- Conversion Rate
- Projected Sales
- ROI
- Risks or challenges
- Final recommendations
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a marketing analyst who simulates campaign performance."},
                {"role": "user", "content": prompt}
            ]
        )
        state.campaign_simulation = response.choices[0].message.content.strip()
    except Exception as e:
        state.campaign_simulation = f"Simulation failed: {str(e)}"

    return state

# 🧾 Agent 7: Export Final Campaign Plan
def export_plan(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal or {}
    filename = f"campaign_report_{parsed.get('campaign_name', 'output').replace(' ', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

    report = f"""
# Final Campaign Report: {parsed.get('campaign_name', 'Unnamed Campaign')}

## 🎯 Goal
{state.goal}

## 📊 Market Analysis
{state.market_analysis}

## 👥 Target Audience
{state.audience_segments}

## 📈 Strategy
{state.campaign_strategy}

## ✍️ Content
{state.campaign_content}

## 🔮 Simulation
{state.campaign_simulation}
"""
    state.campaign_report = report.strip()
    with open(filename, "w", encoding="utf-8") as f:
        f.write(state.campaign_report)
    print(f"📁 Campaign report exported as: {filename}")
    return state

# 🔀 LangGraph Flow
def build_workflow():
    graph = StateGraph(CampaignState)
    graph.add_node("parse_goal", parse_goal)
    graph.add_node("research_data", research_data)
    graph.add_node("segment_audience", segment_audience)
    graph.add_node("plan_strategy", plan_strategy)
    graph.add_node("generate_content", generate_content)
    graph.add_node("simulate_campaign", simulate_campaign)
    graph.add_node("export_plan", export_plan)

    graph.set_entry_point("parse_goal")
    graph.add_edge("parse_goal", "research_data")
    graph.add_edge("research_data", "segment_audience")
    graph.add_edge("segment_audience", "plan_strategy")
    graph.add_edge("plan_strategy", "generate_content")
    graph.add_edge("generate_content", "simulate_campaign")
    graph.add_edge("simulate_campaign", "export_plan")
    graph.set_finish_point("export_plan")
    return graph.compile()

# 🚀 Run the Workflow
def run(goal_text: str):
    embedding_function = OpenAIEmbeddingFunction(api_key=key)
    db_client = chromadb.PersistentClient(path="./chroma_db")
    collection = db_client.get_collection(
        name="campaign_data",
        embedding_function=embedding_function
    )

    state = CampaignState(goal=goal_text, vector_db=collection)
    flow = build_workflow()
    result = flow.invoke(state)

    print("🔎 parsed_goal:", result["parsed_goal"])
    if "error" in result["parsed_goal"]:
        print("Goal parsing failed:", result["parsed_goal"]["error"])
        return

    print("✅ Parsed Goal:\n", result["parsed_goal"])
    print("\n📊 Market Insights from ChromaDB:\n", result["market_analysis"])
    print("\n👥 Target Audience Segment:\n", result["audience_segments"])
    print("\n📈 Final Campaign Strategy:\n", result["campaign_strategy"])
    print("\n✍️ Generated Campaign Content:\n", result["campaign_content"])
    print("\n🔮 Simulated Campaign Performance:\n", result["campaign_simulation"])
    print("\n📁 Report Exported Successfully! Check current directory.")

# ✅ Test It
run("Boost SUV sales in South for the Winter Specials campaign")


📁 Campaign report exported as: campaign_report_Winter_Specials_Campaign_20250408_091059.txt
🔎 parsed_goal: {'goal': 'Boost sales', 'cartype': 'SUV', 'region': 'South', 'campaign_name': 'Winter Specials Campaign'}
✅ Parsed Goal:
 {'goal': 'Boost sales', 'cartype': 'SUV', 'region': 'South', 'campaign_name': 'Winter Specials Campaign'}

📊 Market Insights from ChromaDB:
 Social Media campaign 'Winter Specials' targeting SUVs for Family Buyers offered a 10% discount and had 1284 actual sales.

SMS campaign 'Winter Specials' targeting Coupes for First-Time Buyers offered a 15% discount and had 1213 actual sales.

Email campaign 'Winter Specials' targeting Sedans for Luxury Car Owners offered a 20% discount and had 269 actual sales.

Email campaign 'Winter Specials' targeting Coupes for Luxury Car Owners offered a 5% discount and had 857 actual sales.

Newspaper Ads campaign 'Festive Bonanza' targeting SUVs for Budget Buyers offered a 10% discount and had 1260 actual sales.

👥 Target Audience

#sending emails

In [ ]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

#  Load your updated customer CSV (with email, first_name, last_name)
customers_df = pd.read_csv("/content/customers.csv")

#  Function to send campaign email
def send_campaign_emails(sender_email, app_password, campaign_name, email_subject, email_body_template):
    for _, row in customers_df.iterrows():
        recipient_email = row["email"]
        full_name = f"{row['first_name']} {row['last_name']}"

        # Personalize email body
        personalized_body = email_body_template.replace("{name}", full_name).replace("{campaign}", campaign_name)

        # Compose message
        msg = MIMEMultipart()
        msg["From"] = sender_email
        msg["To"] = recipient_email
        msg["Subject"] = email_subject
        msg.attach(MIMEText(personalized_body, "plain"))

        try:
            # Send via Gmail SMTP
            server = smtplib.SMTP("smtp.gmail.com", 587)
            server.starttls()
            server.login(sender_email, app_password)
            server.sendmail(sender_email, recipient_email, msg.as_string())
            server.quit()
            print(f"✅ Email sent to {full_name} ({recipient_email})")
        except Exception as e:
            print(f"❌ Failed to send email to {recipient_email}: {str(e)}")

# Usage Example
sender_email = "mayureshpawashe3036@gmail.com"
app_password = "rbvskvoolfafsofw"

campaign_name = "Winter Specials"
email_subject = f"🔥 Exclusive Offer - {campaign_name} is Live!"

# Template with placeholder
email_body_template = """
Hi {name},

We're excited to bring you our special {campaign} campaign with up to 15% off on your favorite cars.

Visit our showroom or book online before the offer ends!

🚗 Drive festive joy home today!

Best,
Autonomous Campaign Builder
"""

# Send emails
send_campaign_emails(sender_email, app_password, campaign_name, email_subject, email_body_template)


In [13]:
from pydantic import BaseModel
from typing import Optional, Any
import re
import chromadb
from langgraph.graph import StateGraph
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from google.colab import userdata
from openai import OpenAI
from datetime import datetime
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=key)

# Shared State
class CampaignState(BaseModel):
    goal: str
    parsed_goal: Optional[dict] = None
    vector_db: Optional[Any] = None
    market_analysis: Optional[str] = None
    audience_segments: Optional[str] = None
    campaign_strategy: Optional[str] = None
    campaign_content: Optional[str] = None
    campaign_simulation: Optional[str] = None
    campaign_report: Optional[str] = None

# ✍️ Agent 1: Parse Goal
def parse_goal(state: CampaignState) -> CampaignState:
    text = state.goal
    pattern = re.compile(r"(?i)(boost|increase|grow|improve)\s+(\w+)\s+sales\s+in\s+(\w+)\s+for\s+the\s+(.+)", re.IGNORECASE)
    match = pattern.search(text)
    if match:
        state.parsed_goal = {
            "goal": "Boost sales",
            "cartype": match.group(2).upper(),
            "region": match.group(3).capitalize(),
            "campaign_name": match.group(4).strip().title()
        }
    else:
        state.parsed_goal = {"error": "Unable to parse goal."}
    return state

# 🔍 Agent 2: Research Data from ChromaDB
def research_data(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal
    vector_db = state.vector_db
    if not parsed or not vector_db or "cartype" not in parsed:
        state.market_analysis = "Missing or invalid parsed goal."
        return state
    query = f"{parsed['cartype']} sales and campaign trends in {parsed['region']} for {parsed['campaign_name']} campaign"
    results = vector_db.query(query_texts=[query], n_results=5)
    state.market_analysis = "\n\n".join(results["documents"][0])
    return state

# 👥 Agent 3: Segment Audience
def segment_audience(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal
    vector_db = state.vector_db
    if not parsed or not vector_db or "cartype" not in parsed:
        state.audience_segments = "Missing or invalid parsed goal."
        return state
    query = f"Customers in {parsed['region']} who prefer {parsed['cartype']} cars"
    results = vector_db.query(query_texts=[query], n_results=5)
    state.audience_segments = "\n\n".join(results["documents"][0])
    return state

# 📈 Agent 4: Plan Strategy
def plan_strategy(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal or {}
    research = state.market_analysis
    audience = state.audience_segments
    if not parsed or not research or not audience or "region" not in parsed or "cartype" not in parsed:
        state.campaign_strategy = "Missing inputs for planning strategy."
        return state
    campaign = parsed.get("campaign_name", "Unnamed Campaign")
    region = parsed.get("region", "Unknown")
    cartype = parsed.get("cartype", "Vehicle")
    plan = f"""
📋 Campaign Strategy for '{campaign}':

- 📍 Region: {region}
- 🚗 Cartype: {cartype}
- 📋 Target Audience: Based on segment data, target high-converting buyer groups.
- 📣 Channels: Based on research, use SMS + Email
- 💰 Budget Estimate: ₹10,00,000
- 🍿 Discount: Offer 15% off
- ⏳ Duration: 15 days
- 🎯 KPIs:
  - Sales Target: 150 units
  - Engagement Rate: 25%
    """
    state.campaign_strategy = plan.strip()
    return state

# ✨ Agent 5: Generate Campaign Content
def generate_content(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal
    strategy = state.campaign_strategy
    audience = state.audience_segments

    if not parsed or not strategy or not audience or "cartype" not in parsed:
        state.campaign_content = "Missing data for content generation."
        return state

    try:
        campaign = parsed.get("campaign_name", "Campaign")
        cartype = parsed.get("cartype", "Car")
        region = parsed.get("region", "Your Area")

        content = f"""
📢 Marketing Content for '{campaign}' Campaign:

📱 SMS (3 variants):
1. 🚗 Big Savings on {cartype}s! Get exclusive {campaign} deals in {region}. Limited time!
2. 🔥 {campaign} is live! Grab 15% off on your dream {cartype}. Hurry!
3. 🎉 Special Offer on {cartype}s in {region}! Celebrate with 15% OFF!

✉️ Email:
Subject: Don’t Miss Our {campaign} Deals on {cartype}s!
Body: Celebrate this season with amazing offers on {cartype}s in {region}. Limited-time 15% discount, tailored for you! Visit our showroom or book online now!

📣 Social Media (3 posts):
1. 🎉 #{campaign.replace(" ", "")} – Get 15% OFF on {cartype}s in {region}. Drive festive joy home! 🚗✨
2. 🔥 #{campaign.replace(" ", "")} is here! Unbeatable offers on {cartype}s! Book now.
3. 📢 Exclusive {cartype} discounts for {region} customers. Festival + Fuel = Fun!

🌐 Landing Page:
Headline: "Your {cartype}, Now 15% Off – {campaign} is Here!"
Bullet Points:
- Limited-time festive discount
- Available in {region}
- Top {cartype} brands included
        """
        state.campaign_content = content.strip()
    except Exception as e:
        state.campaign_content = f"Content generation failed: {str(e)}"

    return state

# 🔮 Agent 6: Simulate Campaign Performance
def simulate_campaign(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal
    strategy = state.campaign_strategy
    audience = state.audience_segments
    content = state.campaign_content

    if not parsed or not strategy or not audience or not content:
        state.campaign_simulation = "Simulation skipped: Missing required inputs."
        return state

    prompt = f"""
Based on the following campaign goal and plan, simulate expected marketing performance:

Goal: {parsed}

Strategy:
{strategy}

Audience Insights:
{audience}

Content:
{content}

Give realistic estimates for:
- Estimated Reach
- Engagement Rate
- Conversion Rate
- Projected Sales
- ROI
- Risks or challenges
- Final recommendations
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a marketing analyst who simulates campaign performance."},
                {"role": "user", "content": prompt}
            ]
        )
        state.campaign_simulation = response.choices[0].message.content.strip()
    except Exception as e:
        state.campaign_simulation = f"Simulation failed: {str(e)}"

    return state

# 📤 Email Sending Function
def send_bulk_emails(campaign_name, subject, message_template, sender_email, app_password):
    try:
        customers_df = pd.read_csv("/content/customers.csv")
    except Exception as e:
        print("❌ Could not load customers.csv:", e)
        return

    for _, row in customers_df.iterrows():
        recipient = row["email"]
        name = f"{row['first_name']} {row['last_name']}"
        body = message_template.replace("{name}", name).replace("{campaign}", campaign_name)

        msg = MIMEMultipart()
        msg["From"] = sender_email
        msg["To"] = recipient
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain"))

        try:
            server = smtplib.SMTP("smtp.gmail.com", 587)
            server.starttls()
            server.login(sender_email, app_password)
            server.sendmail(sender_email, recipient, msg.as_string())
            server.quit()
            print(f"✅ Email sent to {name} ({recipient})")
        except Exception as e:
            print(f"❌ Failed to send to {recipient}: {e}")

# 🧾 Agent 7: Export Final Campaign Plan
def export_plan(state: CampaignState) -> CampaignState:
    parsed = state.parsed_goal or {}
    campaign_name = parsed.get("campaign_name", "Unnamed Campaign")
    filename = f"campaign_report_{campaign_name.replace(' ', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

    report = f"""
# Final Campaign Report: {campaign_name}

## 🎯 Goal
{state.goal}

## 📊 Market Analysis
{state.market_analysis}

## 👥 Target Audience
{state.audience_segments}

## 📈 Strategy
{state.campaign_strategy}

## ✍️ Content
{state.campaign_content}

## 🔮 Simulation
{state.campaign_simulation}
"""
    state.campaign_report = report.strip()
    with open(filename, "w", encoding="utf-8") as f:
        f.write(state.campaign_report)
    print(f"📁 Campaign report exported as: {filename}")

    # 📤 Send Emails
    sender_email = "mayureshpawashe3036@gmail.com"  # Replace with your Gmail
    app_password = "rbvskvoolfafsofw"      # Replace with your App Password
    subject = f"🚗 {campaign_name} Campaign - Exclusive Offers!"
    template = f"""
Hi {{name}},

We're thrilled to announce our new campaign: {campaign_name}!

Enjoy exclusive 15% OFF offers on your favorite vehicles during this limited-time campaign.

Explore deals near you and drive away with happiness.

Cheers,
Autonomous Campaign Builder Team
    """
    send_bulk_emails(campaign_name, subject, template, sender_email, app_password)

    return state

# 🔀 LangGraph Flow
def build_workflow():
    graph = StateGraph(CampaignState)
    graph.add_node("parse_goal", parse_goal)
    graph.add_node("research_data", research_data)
    graph.add_node("segment_audience", segment_audience)
    graph.add_node("plan_strategy", plan_strategy)
    graph.add_node("generate_content", generate_content)
    graph.add_node("simulate_campaign", simulate_campaign)
    graph.add_node("export_plan", export_plan)

    graph.set_entry_point("parse_goal")
    graph.add_edge("parse_goal", "research_data")
    graph.add_edge("research_data", "segment_audience")
    graph.add_edge("segment_audience", "plan_strategy")
    graph.add_edge("plan_strategy", "generate_content")
    graph.add_edge("generate_content", "simulate_campaign")
    graph.add_edge("simulate_campaign", "export_plan")
    graph.set_finish_point("export_plan")
    return graph.compile()

# 🚀 Run the Workflow
def run(goal_text: str):
    embedding_function = OpenAIEmbeddingFunction(api_key=key)
    db_client = chromadb.PersistentClient(path="./chroma_db")
    collection = db_client.get_collection(
        name="campaign_data",
        embedding_function=embedding_function
    )

    state = CampaignState(goal=goal_text, vector_db=collection)
    flow = build_workflow()
    result = flow.invoke(state)

    print("🔎 parsed_goal:", result["parsed_goal"])
    if "error" in result["parsed_goal"]:
        print("Goal parsing failed:", result["parsed_goal"]["error"])
        return

    print("✅ Parsed Goal:\n", result["parsed_goal"])
    print("\n📊 Market Insights from ChromaDB:\n", result["market_analysis"])
    print("\n👥 Target Audience Segment:\n", result["audience_segments"])
    print("\n📈 Final Campaign Strategy:\n", result["campaign_strategy"])
    print("\n✍️ Generated Campaign Content:\n", result["campaign_content"])
    print("\n🔮 Simulated Campaign Performance:\n", result["campaign_simulation"])
    print("\n📁 Report Exported Successfully! Check current directory.")

# ✅ Test It
run("Boost SUV sales in South for the Winter Specials campaign")


📁 Campaign report exported as: campaign_report_Winter_Specials_Campaign_20250408_132348.txt
✅ Email sent to Sneha Nair (mayuresh.pawashe@infoobjects.com)
✅ Email sent to Manav Das (chinnu.kurian@infoobjects.com)


KeyboardInterrupt: 